# 使用 Amazon Comprehend 进行主题建模

您可以使用 Amazon Comprehend 检查文档集合的内容以确定主题。例如，您可以为 Amazon Comprehend 提供一系列新闻文章，它将决定文章的主题，例如体育、政治或娱乐。文档中的文本无需进行注释。

Amazon Comprehend 使用隐狄利克雷分布（LDA）以确定一组文档中的主题。它会检查每个文档以确定单词的上下文和含义。

为了获得最准确的结果，您应为 Amazon Comprehend 提供尽可能多的语料库。若要获得最佳效果，建议您：

- 在每个主题建模作业中，应至少使用 1,000 个文档。

- 每个文档应至少有 100 个字符。

- 如果文档主要由数字数据组成，则应将其从语料库中删除。

主题建模是一个异步过程。您可以使用 Amazon S3 存放文本数据，并向 Amazon Comprehend 提交文档列表

注：Amazon Comprehend 主题建模目前还不支持中文

### 导入相关依赖

In [ ]:
import boto3
import time
import pandas as pd

In [ ]:
# 验证S3桶是否已经存在
sts_client = boto3.client("sts")
account_id = sts_client.get_caller_identity()["Account"]
print("Your account id is {}".format(account_id))

bucket_name = "comprehend-labs" + account_id +  "-2"
print ("Bucket name used is " + bucket_name)
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')

if (s3.Bucket(bucket_name).creation_date is None):
    s3_client.create_bucket(Bucket=bucket_name)
    print ("Created bucket " + bucket_name)
else:
    print ("Bucket Exists")

### 查看用于主题建模的文本数据

在Sample.txt中每一行对应一篇文档

In [ ]:
!head Sample.txt

### 创建主题建模任务

In [ ]:
# 上传文本数据至S3
s3 = boto3.resource('s3')
prefix = 'topic_model'

s3.Bucket(bucket_name).upload_file("Sample.txt", "{}/Sample.txt".format(prefix))

In [ ]:
# 获取执行角色
role_name = 'AmazonComprehendServiceRoleS3FullAccess-ComprehendLabs'
iam_client = boto3.client("iam")
response = iam_client.get_role(
    RoleName=role_name
)
comprehend_arn = response['Role']['Arn']
print("The ARN for the role is {}".format(comprehend_arn))

In [ ]:
# 创建主题检测任务
comprehend_client = boto3.client('comprehend')
res = comprehend_client.start_topics_detection_job(
    JobName='news-topic-model',
    InputDataConfig={
        'S3Uri': 's3://{}/{}/Sample.txt'.format(bucket_name, prefix),
        'InputFormat': 'ONE_DOC_PER_LINE'
    },
    OutputDataConfig={
        'S3Uri': 's3://{}/{}'.format(bucket_name, prefix)
    },
    DataAccessRoleArn=comprehend_arn,
    NumberOfTopics=10
)

In [ ]:
# 检查任务是否完成
while True:
    if comprehend_client.describe_topics_detection_job(JobId=res['JobId'])['TopicsDetectionJobProperties']['JobStatus'] == 'IN_PROGRESS':
        print('.', end='')
        time.sleep(3)
    else:
        print('Job done')
        break

### 查看主题建模的结果

In [ ]:
# 下载任务输出的文件
output = comprehend_client.describe_topics_detection_job(JobId=res['JobId'])['TopicsDetectionJobProperties']['OutputDataConfig']['S3Uri']
output_key = output.replace('s3://'+bucket_name+'/', '')
s3.Bucket(bucket_name).download_file(output_key, 'output.tar.gz')

In [ ]:
# 解压并查看结果
!tar -zxvf output.tar.gz

In [ ]:
topic_terms_df = pd.read_csv('topic-terms.csv')
topic_terms_df.head(10)

In [ ]:
doc_topics_df = pd.read_csv('doc-topics.csv')
doc_topics_df.head(10)

- 在topic-terms文件中，我们可以看到和每个topic相关的前十个术语（该数量是创建主题建模任务时，通过NumberOfTopics来指定的）。
- 在doc-topics文件中，我们可以看到不同文本属于topic的比例，可以通过比例大小来决定是否需要给该文本确定预测出来的主题。